In [1]:
import json
import glob
from collections import Counter, defaultdict
import urllib.parse

# Example : Load

In [2]:
filenames = glob.glob('deduplicated_logs/201*-*-*.json')

In [3]:
filename = filenames[0]

In [4]:
with open(filename, 'r') as f:
    data = json.load(f)


# Example : searches by action

In [74]:
data[1]

{'actions': [{'type': 'action',
   'url': 'http://www.data.gouv.fr/fr/datasets/parc-de-logements-1999-2009-en-bretagne-filocom/'},
  {'type': 'action',
   'url': 'http://www.data.gouv.fr/fr/topics/logement-developpement-durable-et-energie/'},
  {'type': 'action',
   'url': 'https://www.data.gouv.fr/fr/topics/logement-developpement-durable-et-energie/datasets'},
  {'type': 'action',
   'url': 'https://www.data.gouv.fr/fr/topics/logement-developpement-durable-et-energie/datasets?page=2'}],
 'date': '2017-12-11',
 'idVisit': '11794423'}

In [71]:
len(data)

10693

In [87]:
regex_datasets = r'^https?://www\.data\.gouv\.fr/(?:fr|en|es)/datasets/([a-z\-0-9]+)/$'

In [90]:
import re
[re.match(regex_datasets, action['url']) for action in data[1]["actions"] if action['type'] == 'action']

[<_sre.SRE_Match object; span=(0, 84), match='http://www.data.gouv.fr/fr/datasets/parc-de-logem>,
 None,
 None,
 None]

In [50]:
for i in range(len(data)):
    actions = data[i]['actions']
    searches = [action for action in actions if action['type'] == 'search']

In [70]:
search_list = []
for i in range(len(data)):
    # On récupère toutes les actions
    actions = data[i]['actions']
    # on ne garde que les actions de type search
    searches = [action for action in actions if action['type'] == 'search']
    if len(searches) > 0:
        s_list.append(searches)

In [64]:
s_list[1]

[{'keyword': 'IR', 'type': 'search', 'url': ''},
 {'keyword': 'IRVE', 'type': 'search', 'url': ''},
 {'keyword': 'RE', 'type': 'search', 'url': ''},
 {'keyword': 'RECHARGE', 'type': 'search', 'url': ''},
 {'keyword': 'RECHAR', 'type': 'search', 'url': ''},
 {'keyword': 'BOR', 'type': 'search', 'url': ''},
 {'keyword': 'AU', 'type': 'search', 'url': ''},
 {'keyword': 'A', 'type': 'search', 'url': ''},
 {'keyword': 'AUBE', 'type': 'search', 'url': ''},
 {'keyword': 'Sarl', 'type': 'search', 'url': ''},
 {'keyword': 'Sarl', 'type': 'search', 'url': ''},
 {'keyword': 'SARL', 'type': 'search', 'url': ''},
 {'keyword': 'Sarl', 'type': 'search', 'url': ''},
 {'keyword': 'SARL', 'type': 'search', 'url': ''},
 {'keyword': 'SARL c', 'type': 'search', 'url': ''},
 {'keyword': 'SARL cp pein', 'type': 'search', 'url': ''},
 {'keyword': 'SARL cp peint', 'type': 'search', 'url': ''},
 {'keyword': 'SARL cp peinture', 'type': 'search', 'url': ''},
 {'keyword': 'SARL cp peinture', 'type': 'search', 'url

In [62]:
goal_list = []
for i in range(len(data)):
    actions = data[i]['actions']
    searches = [action for action in actions if action['type'] == 'goal']
    if len(searches) > 0:
        goal_list.append(searches)

In [63]:
goal_list[1]

[{'type': 'goal',
  'url': 'https://www.data.gouv.fr/fr/reuses/carte-interactive-des-debits-et-de-leligibilite-a-internet-par-commune/'}]

### update:  Il n'y a plus de timestamp dans les actions.

In [ ]:
searches_by_action

# Example : fields by action type

In [37]:
fields_by_type = defaultdict(set)
for i in range(len(data)):
    actions = data[i]['actions']
    for action in actions:
        fields_by_type[action['type']] |= set(action.keys())
fields_by_type

defaultdict(set,
            {'action': {'type', 'url'},
             'download': {'type', 'url'},
             'goal': {'type', 'url'},
             'outlink': {'type', 'url'},
             'search': {'keyword', 'type', 'url'}})

# Features used for search

In [38]:
substring = 'www.data.gouv.fr/fr/datasets/?'

searches = []
for filename in filenames:
    with open(filename, 'r') as f:
        data = json.load(f)
    print('.', end='')

    for i in range(len(data)):
        actions = data[i]['actions']
        searches += [action['url'] for action in actions if action['type'] == 'action' and substring in action['url']]

.......

In [39]:
len(searches)

4547

In [40]:
searches[0:10]

['https://www.data.gouv.fr/fr/datasets/?tag=topographie&granularity=fr:departement',
 'https://www.data.gouv.fr/fr/datasets/?sort=-created&organization=534fffa5a3a7292c64a7809e',
 'https://www.data.gouv.fr/fr/datasets/?sort=-created&organization=534fffa5a3a7292c64a7809e&page=2',
 'https://www.data.gouv.fr/fr/datasets/?sort=-created&organization=534fffa5a3a7292c64a7809e&page=3',
 'https://www.data.gouv.fr/fr/datasets/?sort=-created&organization=534fffa5a3a7292c64a7809e&page=4',
 'http://www.data.gouv.fr/fr/datasets/?organization=534fffa5a3a7292c64a7809e&reuses=many',
 'https://www.data.gouv.fr/fr/datasets/?sort=-created&organization=534fffa5a3a7292c64a7809e&page=4',
 'https://www.data.gouv.fr/fr/datasets/?organization=534fffa5a3a7292c64a7809e&reuses=quite',
 'https://www.data.gouv.fr/fr/datasets/?sort=-followers',
 'https://www.data.gouv.fr/fr/datasets/?sort=-followers&page=2']

In [41]:
feature_count = defaultdict(int)
for search in searches:
    parsed = urllib.parse.urlparse(search)
    features_used = list(urllib.parse.parse_qs(parsed.query).keys())
    for feature in features_used:
        feature_count[feature] += 1

In [42]:
def print_feature_usage(feature_count):
    tuples = list(feature_count.items())
    sorted_tuples = sorted(tuples, key = lambda x: -x[1])
    for feature_name, count in sorted_tuples:
        print('{}\t{}'.format(count, feature_name))

In [43]:
print_feature_usage(feature_count)

1995	page
1715	organization
1444	sort
683	tag
646	geozone
535	format
227	granularity
124	license
108	reuses
62	badge
6	temporal_coverage
1	next


In [91]:
data

[{'actions': [{'type': 'action', 'url': 'http://www.data.gouv.fr/fr/'},
   {'type': 'action',
    'url': 'http://www.data.gouv.fr/fr/topics/sante-et-social/'},
   {'type': 'action',
    'url': 'https://www.data.gouv.fr/fr/datasets/delai-d-attente-rendez-vous-pediatre-0/'},
   {'type': 'download',
    'url': 'https://www.data.gouv.fr/storage/f/2013-12-20T17-38-22/Delai_rdv_pediatres_2012.xlsx'},
   {'type': 'goal',
    'url': 'https://www.data.gouv.fr/fr/datasets/delai-d-attente-rendez-vous-pediatre-0/'},
   {'type': 'action',
    'url': 'http://www.data.gouv.fr/fr/topics/territoires-et-transports/'}],
  'date': '2017-12-11',
  'idVisit': '11794412'},
 {'actions': [{'type': 'action',
    'url': 'http://www.data.gouv.fr/fr/datasets/parc-de-logements-1999-2009-en-bretagne-filocom/'},
   {'type': 'action',
    'url': 'http://www.data.gouv.fr/fr/topics/logement-developpement-durable-et-energie/'},
   {'type': 'action',
    'url': 'https://www.data.gouv.fr/fr/topics/logement-developpement-du